In [ ]:
import pandas as pd


: 

In [1]:
import pandas as pd
import pyspark as ps
sc = ps.SparkContext()
from pyspark.sql import SparkSession


# Create a SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()


22/12/18 21:59:38 WARN Utils: Your hostname, Nikolass-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.3.24.19 instead (on interface en0)
22/12/18 21:59:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/18 21:59:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/18 21:59:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
locationIdToName = "./taxi+_zone_lookup.csv"
taxis = "./taxis"


In [3]:
# create a rdd file from the csv file using pyspark
dff_locactions = spark.read.csv(locationIdToName, header=True, inferSchema=True)
dff_locactions.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [5]:
# create a rdd file from the parquet file using pyspark
dff_taxis = spark.read.parquet(taxis)
dff_taxis.show()



+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2022-03-01 02:13:08|  2022-03-01 02:24:35|            1.0|          2.4|       1.0|                 N|          90|         209|           2|       10.0|  3.0|    0.5|       0.

In [6]:
# create a view for taxis 
dff_taxis.createOrReplaceTempView("taxis")
# do the same for locations
dff_locactions.createOrReplaceTempView("locations")

In [7]:
import time
start = time.time()
# lets filter all taxis trips , that happend in march
marchTrips = spark.sql(
  """
  SELECT *
  FROM taxis 
  INNER JOIN locations ON taxis.DOLocationID = locations.LocationID
  WHERE tpep_pickup_datetime >= '2022-03-01' 
  AND tpep_pickup_datetime < '2022-04-01' and tpep_dropoff_datetime >= '2022-03-01' 
  AND tpep_dropoff_datetime < '2022-04-01'
  AND locations.Zone = "Battery Park"
  ORDER BY Tip_amount DESC
  limit 1
  """
  )

bestTipTripMarch = ((marchTrips).collect())
end = time.time()
print(bestTipTripMarch)
print('Time: ',end - start)

# filter the previus trips that 

[Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2022, 3, 17, 12, 27, 47), tpep_dropoff_datetime=datetime.datetime(2022, 3, 17, 12, 27, 58), passenger_count=1.0, trip_distance=0.0, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=12, DOLocationID=12, payment_type=1, fare_amount=2.5, extra=0.0, mta_tax=0.5, tip_amount=40.0, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=45.8, congestion_surcharge=2.5, airport_fee=0.0, LocationID=12, Borough='Manhattan', Zone='Battery Park', service_zone='Yellow Zone')]
Time:  2.67386794090271


In [12]:
start = time.time()
highestTollFees = spark.sql(
  """
  SELECT MAX(Tolls_amount) as max_toll_fees_per_month, MONTH(tpep_pickup_datetime) as month, 
  MAX(tpep_pickup_datetime) as max_date
  FROM taxis
  where Tolls_amount > 0
  AND YEAR(tpep_pickup_datetime) = '2022'
  GROUP BY MONTH(tpep_pickup_datetime)
  """
  )
highestTollFees = ((highestTollFees).collect())
end = time.time()
print(highestTollFees)
print('Time: ',end - start)

[Row(max_toll_fees_per_month=235.7, month=3, max_date=datetime.datetime(2022, 3, 31, 23, 59, 50)), Row(max_toll_fees_per_month=911.87, month=4, max_date=datetime.datetime(2022, 4, 30, 23, 59, 39)), Row(max_toll_fees_per_month=95.0, month=2, max_date=datetime.datetime(2022, 2, 28, 23, 59, 55)), Row(max_toll_fees_per_month=800.09, month=6, max_date=datetime.datetime(2022, 6, 30, 23, 59, 52)), Row(max_toll_fees_per_month=29.55, month=7, max_date=datetime.datetime(2022, 7, 1, 2, 59, 49)), Row(max_toll_fees_per_month=813.75, month=5, max_date=datetime.datetime(2022, 5, 31, 23, 59, 55)), Row(max_toll_fees_per_month=193.3, month=1, max_date=datetime.datetime(2022, 1, 31, 23, 59, 48))]
Time:  0.680427074432373


# Q3

In [29]:
# group by every 15 days and find the average trip distance and the average trip cost
start = time.time()
avgTripDistanceAndCost = spark.sql(
    """
    SELECT AVG(Trip_distance) as avg_trip_distance, AVG(Total_amount) as avg_trip_cost,
    CASE 
        WHEN 
            EXTRACT(day FROM tpep_pickup_datetime) BETWEEN 1 AND 15 
            THEN DATE_TRUNC('month', tpep_pickup_datetime)
        ELSE DATE_ADD(DATE_TRUNC('month', tpep_pickup_datetime), 15)
    END AS period
    FROM taxis
    WHERE tpep_pickup_datetime >= '2022-01-01'
    AND tpep_pickup_datetime < '2022-12-31'
    GROUP BY period
    ORDER BY period DESC
    """
    )
avgTripDistanceAndCost = ((avgTripDistanceAndCost).collect())
end = time.time()
print(avgTripDistanceAndCost)
print('Time: ',end - start)

[Row(avg_trip_distance=3.8276608700743653, avg_trip_cost=21.701845541305357, period=datetime.datetime(2022, 7, 1, 0, 0)), Row(avg_trip_distance=5.9169746181557965, avg_trip_cost=22.067763372211772, period=datetime.datetime(2022, 6, 16, 0, 0)), Row(avg_trip_distance=6.0802899985688335, avg_trip_cost=22.172336874148503, period=datetime.datetime(2022, 6, 1, 0, 0)), Row(avg_trip_distance=7.569369040277797, avg_trip_cost=22.491229166374467, period=datetime.datetime(2022, 5, 16, 0, 0)), Row(avg_trip_distance=6.057147110233409, avg_trip_cost=21.63975255180099, period=datetime.datetime(2022, 5, 1, 0, 0)), Row(avg_trip_distance=5.776266087061577, avg_trip_cost=21.163756511111103, period=datetime.datetime(2022, 4, 16, 0, 0)), Row(avg_trip_distance=5.569090353516405, avg_trip_cost=21.22049664355691, period=datetime.datetime(2022, 4, 1, 0, 0)), Row(avg_trip_distance=5.377357732415769, avg_trip_cost=20.827389869581747, period=datetime.datetime(2022, 3, 16, 0, 0)), Row(avg_trip_distance=6.1982732236

In [30]:
rdd_taxis = dff_taxis.rdd
rdd_locations = dff_locactions.rdd
print(rdd_locations)

MapPartitionsRDD[139] at javaToPython at DirectMethodHandleAccessor.java:104


In [34]:
from datetime import datetime 
def get_period(dt):
    day = dt.day
    if day >= 1 and day <= 15:
        return datetime(dt.year, dt.month, 1)
    else:
        return datetime(dt.year, dt.month, 16)

In [35]:
# do the same but in rdd format
start = time.time()
avgTripDistanceAndCostRdd = rdd_taxis.filter(lambda x: [0] )

avgTripDistanceAndCostRdd = avgTripDistanceAndCostRdd.reduce(lambda x, y: x + y)
end = time.time()
print(avgTripDistanceAndCostRdd)
print('Time: ',end - start)

ERROR:root:KeyboardInterrupt while sending command.][Stage 65:>   (0 + 0) / 8]
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/homebrew/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/homebrew/Cellar/python@3.10/3.10.9/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 